<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### MVP Portfolio Analysis Class

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Standardizing Tasks

Topics of interest include:

* Importing, visualizing the data
* `FinancialData` base class
* `MVPPortfolio` portfolio class

## Real Data

**_Historical end-of-day financial time series data._**

Data retrieved vom Refinitiv Eikon for the German DAX 30 constituents.

The data sets:

    http://hilpisch.com/dax_eikon_eod_data.csv
    http://hilpisch.com/dax_eikon_mc_data.csv

## Imports and Data

In [ ]:
!git clone https://github.com/tpq-classes/python_for_asset_management.git
import sys
sys.path.append('python_for_asset_management')


In [ ]:
from fix_cufflink import *
cf_colors.to_rgba = fixed_to_rgba
cf_plotlytools.to_rgba = fixed_to_rgba

In [ ]:
!pip install eikon

In [ ]:

import math
import cufflinks
import eikon as ek
import numpy as np
import pandas as pd
import plotly.io as pio
from pylab import plt
plt.style.use('seaborn-v0_8')
cufflinks.go_offline()
cufflinks.set_config_file(offline=True)
np.set_printoptions(suppress=True, precision=4)
pd.options.display.float_format = '{:.5f}'.format
%config InlineBackend.figure_format = 'svg'
pio.renderers.default = "colab"


In [ ]:
url = 'http://hilpisch.com/dax_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0)

In [ ]:
raw.iloc[:5, :5]

In [ ]:
universe = raw.columns# [-10:]
universe

## Financial Data Class

Methods to be implemented:

    .__init__()
    .retrieve_data()
    .prepare_data()
    .plot_data()
    .plot_mc()
    .plot_corr()

In [ ]:
class FinancialData:
    url = 'http://hilpisch.com/dax_eikon_eod_data.csv'
    url_ = 'http://hilpisch.com/dax_eikon_mc_data.csv'
    def __init__(self, universe):
        self.universe = universe
        self.no_assets = len(universe)
        self.retrieve_data()
        self.prepare_data()
    def retrieve_data(self):
        self.raw = pd.read_csv(self.url, index_col=0)
        self.raw_ = pd.read_csv(self.url_, index_col=0)
    def prepare_data(self):
        self.data = self.raw[self.universe]
        self.rets = np.log(self.data / self.data.shift(1))
        self.mc = (self.raw_.T[self.universe]).T
        self.mc['MC%'] = self.mc['MC'].apply(lambda x: x / self.mc['MC'].sum())
    def plot_data(self, cols=None):
        if cols is None:
            cols = self.universe
        self.data[cols].normalize().iplot()
    def plot_mc(self):
        self.mc.sort_values('MC').iplot(kind='pie',
                values='MC', labels='NAME', colorscale='rdylbu')
    def plot_corr(self):
        self.rets.corr().iplot(kind='heatmap', colorscale='reds')

In [ ]:
fd = FinancialData(universe)

In [ ]:
# fd.raw

In [ ]:
# fd.raw_.T

In [ ]:
# fd.mc

In [ ]:
fd.plot_data()

In [ ]:
fd.plot_mc()

In [ ]:
fd.plot_corr()

## MVP Class

Methods to be implemented:

    .__init__()
    .portfolio_return()
    .portfolio_variance()
    .portfolio_volatility()
    .portfolio_sharpe()
    ._set_bounds_constraints()
    ._get_results()
    .minimum_volatility_portfolio()
    .maximum_sharpe_portfolio()
    .plot_weights()
    .plot_performance()

In [ ]:
universe = raw.columns[-10:]
universe

In [ ]:
from scipy.optimize import minimize

In [ ]:
class MVPPortfolio(FinancialData):
    def __init__(self, universe):
        super().__init__(universe)
        self.equal_weights = self.no_assets * [1 / self.no_assets]
        self.mc_weights = self.mc['MC%'].values
    def portfolio_return(self, weights, days=252):
        return np.dot(self.rets.mean(), weights) * days
    def portfolio_variance(self, weights, days=252):
        return np.dot(weights, np.dot(self.rets.cov(), weights)) * days
    def portfolio_volatility(self, weights, days=252):
        return math.sqrt(self.portfolio_variance(weights, days))
    def portfolio_sharpe(self, weights, days=252):
        sharpe = (self.portfolio_return(weights, days) /
                  self.portfolio_volatility(weights, days))
        return sharpe
    def minimum_volatility_portfolio(self, bnds=None, cons=None):
        if bnds is None:
            bnds = self.no_assets * [(0, 1)]
        if cons is None:
            cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}
        opt = minimize(self.portfolio_volatility, self.equal_weights,
                      bounds=bnds, constraints=cons)
        return opt
    def maximum_sharpe_portfolio(self, bnds=None, cons=None):
        if bnds is None:
            bnds = self.no_assets * [(0, 1)]
        if cons is None:
            cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}
        tf = lambda weights: -self.portfolio_sharpe(weights)
        opt = minimize(tf, self.equal_weights, bounds=bnds, constraints=cons)
        return opt

In [ ]:
mvp = MVPPortfolio(universe)

In [ ]:
mvp.equal_weights

In [ ]:
mvp.portfolio_return(mvp.equal_weights)

In [ ]:
mvp.portfolio_volatility(mvp.equal_weights)

In [ ]:
mvp.portfolio_sharpe(mvp.equal_weights)

In [ ]:
mvp.mc_weights

In [ ]:
mvp.portfolio_return(mvp.mc_weights)

In [ ]:
mvp.portfolio_volatility(mvp.mc_weights)

In [ ]:
mvp.portfolio_sharpe(mvp.mc_weights)

In [ ]:
mvp.minimum_volatility_portfolio()

In [ ]:
mvp.maximum_sharpe_portfolio(bnds=None)

### Minimum Volatility

### Maximum Sharpe Ratio 

### Bounded Maximum Sharpe

### Short Sales Allowed

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>